In [5]:
import torch
from scipy.io import wavfile
import torch.utils.data as Data
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [6]:
sample_rate_original,waveform_original= wavfile.read('cello_1_min.wav')
_,waveform_1  = wavfile.read('cello_1_min_delay_1.wav')
_,waveform_2 = wavfile.read('cello_1_min_delay_2.wav')
_,waveform_3 = wavfile.read('cello_1_min_long_delayed.wav')
_,waveform_4 = wavfile.read('cello_1_min_noise_mid.wav')
_,waveform_5 = wavfile.read('cello_1_min_noise_start.wav')
_,test_audio = wavfile.read('test.wav') 
waveform_original = waveform_original.astype('float32')
waveform_original /= 255
print("Shape of waveform: {}".format(np.shape(waveform_original)))
print("Sample rate of waveform: {}".format(sample_rate_original))
print(np.shape(waveform_1))

C:\Users\darshan19\anaconda3\envs\AoI\lib\site-packages\ipykernel_launcher.py:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  """Entry point for launching an IPython kernel.
C:\Users\darshan19\anaconda3\envs\AoI\lib\site-packages\ipykernel_launcher.py:2: WavFileWarning: Chunk (non-data) not understood, skipping it.
  
C:\Users\darshan19\anaconda3\envs\AoI\lib\site-packages\ipykernel_launcher.py:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\darshan19\anaconda3\envs\AoI\lib\site-packages\ipykernel_launcher.py:4: WavFileWarning: Chunk (non-data) not understood, skipping it.
  after removing the cwd from sys.path.
C:\Users\darshan19\anaconda3\envs\AoI\lib\site-packages\ipykernel_launcher.py:5: WavFileWarning: Chunk (non-data) not understood, skipping it.
  """
C:\Users\darshan19\anaconda3\envs\AoI\lib\site-packages\ipykernel_launcher.py:6: WavFileWarning: Chunk (

Shape of waveform: (2867795, 2)
Sample rate of waveform: 44100
(2867795, 2)


In [7]:
samples=100
BATCH_SIZE = 20
waveform_1 = waveform_3.astype('float32')
waveform_1 /= 255
waveform_2 = waveform_3.astype('float32')
waveform_2 /= 255
waveform_3 = waveform_3.astype('float32')
waveform_3 /= 255
waveform_4 = waveform_3.astype('float32')
waveform_4 /= 255
waveform_5 = waveform_3.astype('float32')
waveform_5 /= 255
test_audio = test_audio.astype('float32')
test_audio /=255

waveform_original = np.repeat(torch.tensor(np.transpose(waveform_original))[np.newaxis,...], BATCH_SIZE, axis=0)
y1 = np.repeat(torch.tensor(np.transpose(waveform_1))[np.newaxis,...], samples, axis=0)
y2 = np.repeat(torch.tensor(np.transpose(waveform_2))[np.newaxis,...], samples, axis=0)
y3 = np.repeat(torch.tensor(np.transpose(waveform_3))[np.newaxis,...], samples, axis=0)
y4 = np.repeat(torch.tensor(np.transpose(waveform_4))[np.newaxis,...], samples, axis=0)
y5 = np.repeat(torch.tensor(np.transpose(waveform_5))[np.newaxis,...], samples, axis=0)

y = np.concatenate((y1,y2,y3,y4,y5),axis=0)
print(np.shape(waveform_original))


np.random.shuffle(y)



print(np.shape(y3))
print(y3)

torch.Size([20, 2, 2867795])
torch.Size([100, 2, 2867795])
tensor([[[-0.0078,  0.0118, -0.0157,  ..., -3.6078, -3.7608, -3.8902],
         [ 0.0118, -0.0157,  0.0157,  ..., -3.7725, -3.8902, -3.9608]],

        [[-0.0078,  0.0118, -0.0157,  ..., -3.6078, -3.7608, -3.8902],
         [ 0.0118, -0.0157,  0.0157,  ..., -3.7725, -3.8902, -3.9608]],

        [[-0.0078,  0.0118, -0.0157,  ..., -3.6078, -3.7608, -3.8902],
         [ 0.0118, -0.0157,  0.0157,  ..., -3.7725, -3.8902, -3.9608]],

        ...,

        [[-0.0078,  0.0118, -0.0157,  ..., -3.6078, -3.7608, -3.8902],
         [ 0.0118, -0.0157,  0.0157,  ..., -3.7725, -3.8902, -3.9608]],

        [[-0.0078,  0.0118, -0.0157,  ..., -3.6078, -3.7608, -3.8902],
         [ 0.0118, -0.0157,  0.0157,  ..., -3.7725, -3.8902, -3.9608]],

        [[-0.0078,  0.0118, -0.0157,  ..., -3.6078, -3.7608, -3.8902],
         [ 0.0118, -0.0157,  0.0157,  ..., -3.7725, -3.8902, -3.9608]]])


In [8]:
torch.manual_seed(1)

  # size of the minibatch
#z = torch.from_numpy(y)
# load dataset into dataLoader
#print(type(z))
loader = Data.DataLoader(

    dataset=y3,

    batch_size=BATCH_SIZE,

    shuffle=True,

    num_workers=0,

)



class DiscreteNet(torch.nn.Module):
        # use an embedding layer to build a NN

    def __init__(self):

        super(DiscreteNet, self).__init__()

        self.embedding = nn.LSTM(2867795,2)
        self.embedding2 = nn.Linear(2,2867795)


    def forward(self, x):

        y,x = self.embedding(x)
        y = self.embedding2(y)
        return y



f = DiscreteNet()


optimizer = optim.Adam(params=f.parameters(), lr=0.01)
loss = nn.MSELoss()


def run():
    for epoch in range(500):

        for step,(batch_x) in enumerate(loader):
            optimizer.zero_grad()
            print(np.shape(batch_x))
            fx = f(batch_x)
            print(np.shape(fx),np.shape(waveform_original))
            l = loss(fx,waveform_original)
            l.backward()
            optimizer.step()
            print("Epoch: ",epoch,"Loss: ",l.item())

    print('Finished Training')


run()

torch.Size([20, 2, 2867795])
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  0 Loss:  94.85454559326172
torch.Size([20, 2, 2867795])
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  0 Loss:  94.7296371459961
torch.Size([20, 2, 2867795])
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  0 Loss:  94.60490417480469
torch.Size([20, 2, 2867795])
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  0 Loss:  94.4802474975586
torch.Size([20, 2, 2867795])
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  0 Loss:  94.35600280761719
torch.Size([20, 2, 2867795])
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  1 Loss:  94.23188018798828
torch.Size([20, 2, 2867795])
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  1 Loss:  94.10807800292969
torch.Size([20, 2, 2867795])
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  1 Loss:  93.98442840576172
torch.Size([20, 2, 2867795

KeyboardInterrupt: 

In [4]:
test_audio = torch.tensor(test_audio)
d = f(test_audio.reshape(1,2,2867795))
d = d.reshape(2, 2867795)
wavfile.write('reconstructed_test.wav',sample_rate_original,np.transpose(d.detach().numpy()))



C:\Users\darshan19\anaconda3\envs\AoI\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  """Entry point for launching an IPython kernel.


NameError: name 'f' is not defined

In [16]:
from sklearn.metrics import mean_squared_error 


# Calculation of Mean Squared Error (MSE) 
mean_squared_error(waveform_original[0].detach().numpy(),d.detach().numpy()) 


24.543398